In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
# Import API key
import gmaps
# Google developer API key
from datetime import date
from config import gkey
from pprint import pprint
# Configure gmaps
gmaps.configure(api_key=gkey)


In [2]:
#data provided by emily and harlan https://github.com/NABSA/gbfs/blob/master/systems.csv
bigdata_df=pd.read_csv("big_data.csv")
bigdata_df


,Country Code,Name,Location,System ID,URL,Auto-Discovery URL
0,AE,ADCB Bikeshare,"Abu Dhabi, AE",ABU,https://www.bikeshare.ae/,https://api-core.bikeshare.ae/gbfs/gbfs.json
1,AE,Careem BIKE,"Dubai, AE",careem_bike,https://www.careem.com/en-ae/careem-bike/,https://dubai.publicbikesystem.net/ube/gbfs/v1/
2,AR,Ecobici,"Buenos Aires, AR",bike_buenosaires,https://www.buenosaires.gob.ar/ecobici,https://buenosaires.publicbikesystem.net/ube/g...
3,AR,MiBiciTuBici,"Rosario, Santa Fe, AR",biketobike,https://www.mibicitubici.gob.ar/,https://www.mibicitubici.gob.ar/opendata/gbfs....
4,AR,Movo Buenos Aires,"Buenos Aires, AR",movo_bue,https://movo.me/ar,https://gbfs.movo.me/v1/buenosaires/gbfs.json
...,...,...,...,...,...,...
530,US,Wheels Miami,"Miami, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://mia-gbfs.getwheelsapp.com/gbfs.json
531,US,Wheels Orlando,"Orlando, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://orlando-gbfs.getwheelsapp.com/gbfs.json
532,US,Wheels San Diego,"San Diego, CA",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://sandiego-gbfs.getwheelsapp.com/gbfs.json
533,US,Wheels Tallahassee,"Tallahassee, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://tallahassee-gbfs.getwheelsapp.com/gbfs...


In [3]:
#getting data from only US cities
us_data=bigdata_df.loc[(bigdata_df["Country Code"]=="US")]
us_data

,Country Code,Name,Location,System ID,URL,Auto-Discovery URL
331,US,Austin B-cycle,"Austin, TX",bcycle_austin,http://austinbcycle.com,https://gbfs.bcycle.com/bcycle_austin/gbfs.json
332,US,Aventura BCycle,"Aventura, FL",bcycle_aventura,https://aventura.bcycle.com,https://gbfs.bcycle.com/bcycle_aventura/gbfs.json
333,US,Bay Wheels,"San Francisco Bay Area, CA",BA,https://www.baywheels.com/,https://gbfs.baywheels.com/gbfs/gbfs.json
334,US,Bike Chattanooga,"Chattanooga, TN",bike_chattanooga,http://www.bikechattanooga.com/,https://chat.publicbikesystem.net/ube/gbfs/v1/
335,US,BikeLNK,"Lincoln, NE",bcycle_bikelnk,https://bikelnk.bcycle.com,https://gbfs.bcycle.com/bcycle_bikelnk/gbfs.json
...,...,...,...,...,...,...
530,US,Wheels Miami,"Miami, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://mia-gbfs.getwheelsapp.com/gbfs.json
531,US,Wheels Orlando,"Orlando, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://orlando-gbfs.getwheelsapp.com/gbfs.json
532,US,Wheels San Diego,"San Diego, CA",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://sandiego-gbfs.getwheelsapp.com/gbfs.json
533,US,Wheels Tallahassee,"Tallahassee, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://tallahassee-gbfs.getwheelsapp.com/gbfs...


In [5]:
#getting data for bird and lime escooter
data_scoot = us_data.loc[us_data['System ID'].str.contains("lime", case=False)]
len(data_scoot)
#data_scoot
data_scoot

,Country Code,Name,Location,System ID,URL,Auto-Discovery URL
398,US,Lime Arlington,"Arlington, VA",lime_arlington,https://www.li.me/,https://data.lime.bike/api/partners/v1/gbfs/ar...
399,US,Lime Baltimore,"Baltimore, MD",lime_baltimore,https://www.li.me/,https://data.lime.bike/api/partners/v1/gbfs/ba...
400,US,Lime Chicago,"Chicago, IL",lime_chicago,https://www.li.me/,https://data.lime.bike/api/partners/v1/gbfs/ch...
401,US,Lime Cleveland,"Cleveland, OH",lime_cleveland,https://li.me/,https://data.lime.bike/api/partners/v1/gbfs/cl...
402,US,Lime Detroit,"Detroit, MI",lime_detroit,https://www.li.me/,https://data.lime.bike/api/partners/v1/gbfs/de...
403,US,Lime Louisville,"Louisville, KY",lime_louisville,https://www.li.me/,https://data.lime.bike/api/partners/v1/gbfs/lo...
404,US,Lime New York,"New York, NY",lime_new_york,https://www.li.me/,https://data.lime.bike/api/partners/v1/gbfs/ne...
405,US,Lime Oakland,"Oakland, CA",lime_oakland,https://www.li.me/,https://data.lime.bike/api/partners/v1/gbfs/oa...
406,US,Lime Portland,"Portland, OR",lime_portland,https://www.li.me/,https://data.lime.bike/api/partners/v1/gbfs/po...
407,US,Lime Providence,"Providence, RI",lime_providence,https://www.li.me,https://data.lime.bike/api/partners/v1/gbfs/pr...


In [6]:
# test_url = ["https://gbfs.spin.pm/api/gbfs/v1/washington_dc/gbfs",
#        "https://gbfs.spin.pm/api/gbfs/v1/san_francisco/gbfs", 
#        "https://gbfs.spin.pm/api/gbfs/v1/tampa/gbfs"]


In [19]:
# # TESTING HOW ITERROWS WORK - WORKED
# x=1
# for i, row in data_scoot.head(5).iterrows():
# #     unique_id = i
#     city = row['Location']
#     url = row['Auto-Discovery URL']
#     print(city)
#     print(url)
#     x=x+1
# print(x)


In [7]:
# #getting latitudes and longitudes for city as well as coordinates of bird scooters in each cities
# coord_length=[]
lat_list=[]
lon_list=[]
latlon = []
all_url=data_scoot["Auto-Discovery URL"]
x = 0
y = 0
bike = 0
city_list = []
dict = {}
city_list_coord = []
count_list = []
for i, row in data_scoot.iterrows():
        
    try:
        # print(base_url)
        base_url= row['Auto-Discovery URL']
        cities = row['Location']
        print(f'running record {x} on {cities}')

        # get URL for location
        result = requests.get(base_url).json()
        result
        # Fixed the jsondecode issue
        #response = requests.get(base_url).json()
        # print(json.dumps(response, indent=4, sort_keys=True))


        # get URL with bike status for location
        base_url1 = result['data']['en']['feeds'][3]['url']
        base_url1

        # get file with location info
        result1 = requests.get(base_url1).json()
        result1

        bike_coord = result1['data']['bikes']
        bike_coord
        print(f'there are {len(bike_coord)} bikes in record {x}')
        city_list.append(cities)
        count_list.append(len(bike_coord))
        
        for bike in bike_coord:

            # Add Lat info to list
            lat = [bike][0]['lat']
            lon = [bike][0]['lon']
            stripped_lat = lat.strip('')
            stripped_lon = lon.strip('')
    #         print(stripped_lat)
    #         print(stripped_lon)
            lat_list.append(float(stripped_lat))
            lon_list.append(float(stripped_lon))
            city_list_coord.append(cities)

    except ValueError:
        print("Decoding JSON has failed - no bike available")
        city_list.append(cities)
        count_list.append(0)
        
    x = x+1
    time.sleep(2)
 
# print(lat_list)
# print(lon_list)
# # # latlon.append(dict)
# # #         x = x +1
# # #         y = y+1
        

        

running record 0 on Arlington, VA
there are 272 bikes in record 0
running record 1 on Baltimore, MD
there are 528 bikes in record 1
running record 2 on Chicago, IL
Decoding JSON has failed - no bike available
running record 3 on Cleveland, OH
Decoding JSON has failed - no bike available
running record 4 on Detroit, MI
Decoding JSON has failed - no bike available
running record 5 on Louisville, KY
Decoding JSON has failed - no bike available
running record 6 on New York, NY
Decoding JSON has failed - no bike available
running record 7 on Oakland, CA
Decoding JSON has failed - no bike available
running record 8 on Portland, OR
Decoding JSON has failed - no bike available
running record 9 on Providence, RI
Decoding JSON has failed - no bike available
running record 10 on San Francisco, CA
Decoding JSON has failed - no bike available
running record 11 on Seattle, WA
Decoding JSON has failed - no bike available
running record 12 on Washington, DC
Decoding JSON has failed - no bike available

In [17]:
#pull for one city
lat_list = []
lon_list = []

base_url= "https://gbfs.spin.pm/api/gbfs/v1/washington_dc/gbfs"
city = "WashingtonDC"
data = requests.get(base_url).json()
base_url1 = data['data']['en']['feeds'][3]['url']
result1 = requests.get(base_url1).json()
bike_coord = result1['data']['bikes']
print(f'There are {len(bike_coord)} bikes in {city}.')
for bike in bike_coord:

            # Add Lat info to list
            lat = [bike][0]['lat']
            lon = [bike][0]['lon']
            stripped_lat = lat.strip('')
            stripped_lon = lon.strip('')
    #         print(stripped_lat)
    #         print(stripped_lon)
            lat_list.append(float(stripped_lat))
            lon_list.append(float(stripped_lon))
            

IndexError: list index out of range

In [16]:
#Save csv for one city
city_bird_df = pd.DataFrame({'Lat':lat_list, 'Lon': lon_list})
city_bird_df['Operator'] = "Lime"
today = date.today()
city_bird_df['Date'] = today
city_bird_df.to_csv('SanFrancisco_Lime_Lat_Lon.csv')

In [20]:
#To save geocode info to csv
df1 = pd.DataFrame({'City': city_list, 'Count':count_list})
df2 = pd.DataFrame({'Lat':lat_list, 'Lon': lon_list, 'City': city_list_coord})
df2['Operator'] = "Lime"
today = date.today()
df2['Date'] = today
# print(len(lat_list))
# print(len(city_list_coord))
# print(len(lon_list))
# print(len(city_list))
# print(len(count_list))
df2
df2.to_csv('Geocode_Lime_Lat_Lon.csv')


In [17]:
locations = list(zip(lat_list,lon_list))
locations
len(locations)

800

In [18]:

#generating the heat map using the city(lat and lon values) with
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, dissipating=False, max_intensity=100,
                                point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [45]:
#look at google gmaps documentation
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Operator Name</dt><dd>{Operator}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
escooter_info = [info_box_template.format(**row) for index, row in df.iterrows()]
locations = df[["Lat", "Lon"]]

In [46]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=escooter_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))